In [1]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import os
import json
import urllib
import unicodedata
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

#General stuff

In [2]:
CACHE_FOLDER = "cache"
PLAIN_TEXT_FOLDER = "plain_text"

In [3]:
def load_content(page, params=None):
    if not os.path.exists(CACHE_FOLDER):
        os.makedirs(CACHE_FOLDER)

    # Create a session with connection pooling
    session = requests.Session()

    # Define the number of retries and backoff factor
    retries = Retry(total=5, backoff_factor=0.1, status_forcelist=[500, 502, 503, 504])

    # Mount the HTTPAdapter with the Retry settings to the session
    session.mount('http://', HTTPAdapter(max_retries=retries))
    session.mount('https://', HTTPAdapter(max_retries=retries))

    #load content
    content = ""
    cache_location = os.path.join(CACHE_FOLDER,page.split('/')[-1].replace('/', '_'))
    if params is not None:
        params_hash = hash(frozenset(params.items()))
        cache_location += "_" + str(params_hash)
    if not os.path.exists(cache_location):
        result = session.get(page, allow_redirects=True, params=params)
        content = result.text
        with open(cache_location, 'w') as f:
            f.write(content)
    else:
        with open(cache_location) as f:
            content = f.read()
    return content

In [4]:
def slugify(value, allow_unicode=False):
    """
    Taken from https://github.com/django/django/blob/master/django/utils/text.py
    Convert to ASCII if 'allow_unicode' is False. Convert spaces or repeated
    dashes to single dashes. Remove characters that aren't alphanumerics,
    underscores, or hyphens. Convert to lowercase. Also strip leading and
    trailing whitespace, dashes, and underscores.
    """
    value = str(value)
    if allow_unicode:
        value = unicodedata.normalize('NFKC', value)
    else:
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
    value = re.sub(r'[^\w\s-]', '', value.lower())
    return re.sub(r'[-\s]+', '-', value).strip('-_')

#Scrape all hurraki data

In [5]:
URL_ALL_PAGES = "https://hurraki.de/w/api.php?action=query&format=json&list=allpages&aplimit=500&apfilterredir=nonredirects&apfrom={}"
URL_GET_PAGE = "https://hurraki.de/w/api.php?action=parse&prop=categories|text&format=json&page={}"

In [6]:
def get_all_titles():

  apfrom = "A"
  all_pages = []

  #iterate over all pages (max. 500 entries per page)
  while apfrom is not None:

    #load data
    print(load_content(URL_ALL_PAGES.format(apfrom)))
    data = json.loads(load_content(URL_ALL_PAGES.format(apfrom)))
    pages = data["query"]["allpages"]

    for page in pages:
      if page["title"].startswith("Hurraki:"):
        continue
      all_pages.append(page["title"])

    if "continue" in data.keys():
      apfrom = data['continue']['apcontinue']
    else:
      apfrom = None

  return all_pages

all_titles = get_all_titles()

{"batchcomplete":"","continue":{"apcontinue":"Bollenhut","continue":"-||"},"query":{"allpages":[{"pageid":11053,"ns":0,"title":"ABBA"},{"pageid":3744,"ns":0,"title":"ADAC"},{"pageid":3122,"ns":0,"title":"ADHS"},{"pageid":2665,"ns":0,"title":"AFAIK"},{"pageid":3141,"ns":0,"title":"AFK"},{"pageid":7747,"ns":0,"title":"AN-12 Unfall in Juba"},{"pageid":10221,"ns":0,"title":"Aarhus-Konvention"},{"pageid":1202,"ns":0,"title":"Aaron Fotheringham"},{"pageid":5975,"ns":0,"title":"Abbruchkante"},{"pageid":10136,"ns":0,"title":"Abendland"},{"pageid":4811,"ns":0,"title":"Abendrot"},{"pageid":8278,"ns":0,"title":"Abfall"},{"pageid":8275,"ns":0,"title":"Abfalltrennung"},{"pageid":10856,"ns":0,"title":"Abfindung"},{"pageid":10691,"ns":0,"title":"Abf\u00fchren"},{"pageid":10690,"ns":0,"title":"Abf\u00fcllen"},{"pageid":929,"ns":0,"title":"Abgeordnete"},{"pageid":1713,"ns":0,"title":"Abgeordnetengesetz"},{"pageid":11942,"ns":0,"title":"Abhol\u00b7zeit"},{"pageid":7424,"ns":0,"title":"Abh\u00f6ren"},{"p

In [7]:
full_links = [URL_GET_PAGE.format(urllib.parse.quote(sub)) for sub in all_titles]

slugified_titles = [slugify(title) for title in all_titles]
link_dataframe = pd.DataFrame(data={'title':slugified_titles,'link':full_links})
link_dataframe.to_csv('meta.csv', index=False)

In [8]:
#skip previous cells if you allready scraped the links
link_dataframe = pd.read_csv('meta.csv', keep_default_na=False)

In [50]:
#Define specific processing

IMAGE_CAPTION = re.compile(r"Auf dem Bild ist *? zu sehen")
DUPLICATED_NEW_LINES = re.compile(r"(?<=\n\n)\s*\n")
UNIFORM_BULLET_POINTS = re.compile(r"(?<=\n)\s*- *")

REMOVE = {
  IMAGE_CAPTION:'',
  DUPLICATED_NEW_LINES:'',
  UNIFORM_BULLET_POINTS:'• ',
}

def process_text(text):
    for remove_item, replacement in REMOVE.items():
        text = remove_item.sub(replacement, text)

    return text.strip()

In [59]:
def to_plain_text(html):
    #print(html)
    LONG_DESCRIPTION_SEPARATOR = """<span class="mw-headline" id="Genaue_Erklärung">Genaue Erklärung</span>"""

    #split sections
    splitted = html.split("<span class=\"mw-headline\"")
    #reconstruct sections
    for i in range(1,len(splitted)):
        splitted[i] = "<span class=\"mw-headline\"" + splitted[i]

    long_description_html = ""
    short_description_html = splitted[0]
    
    for section in splitted:
        if LONG_DESCRIPTION_SEPARATOR in section:
            long_description_html = section
            break

    text = short_description_html #+ "\n" + long_description_html
    soup = BeautifulSoup(text, "html.parser")

    #format lists to recognize them with regex
    for ul_list in soup.find_all('ul') + soup.find_all('ol'):

        for item in ul_list.find_all('li'):
            if item.text.strip() != "":
                item.string = "• " + item.text.strip()

    for span in soup.find_all('span', class_='mw-editsection'):
        span.clear()
        
    for table in soup.find_all("table"):
        table.clear()
        
    # Extract text without HTML tags
    plain_text = soup.get_text(separator=' ', strip=True)
    
    plain_text = process_text(plain_text)
    
    def get_first_two_sentences(text):
        sentences = re.split(r"(?<!\w\.\w.)(?<![0-9]\.)(?<![0-9][0-9]\.)(?<![A-Z]\.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s", text)
        sentences = [s for s in sentences if s]

        if len(sentences) >= 2:
            return ' '.join(sentences[:2])
        elif len(sentences) == 1:
            return sentences[0]
        else:
            return ""
    
    final_text = get_first_two_sentences(plain_text)
    print(final_text)

    return final_text

In [60]:
def load_and_save_plain_text(link, name=None):

    result = json.loads(load_content(link))
    if 'parse' not in result:
        print(link)
        categories = []
        return
    content = result['parse']['text']['*']
    categories = []
    for item in result['parse']['categories']:
        categories.append(item['*'])
    category = "|".join(categories)

    if name is None:
        name = link.split('/')[-1]

    name = name.replace('/','_') + '.txt'

    #process article
    plain_text = to_plain_text(content)

    if not os.path.exists(PLAIN_TEXT_FOLDER):
        os.makedirs(PLAIN_TEXT_FOLDER)

    #save plain text
    with open(os.path.join(PLAIN_TEXT_FOLDER , name), 'w+') as f:
        f.write(plain_text)

    return {'genre': category}

In [61]:
#load and parse all pages

exclude_pages = ['Übung: Demoversion',
                 'Übung: jaki',
                 'Übung: jan poka',
                 'Übung: kala 🐟',
                 'Übung: kili 🍎',
                 'Übung: kulupu tomo','Übung: ma Apika',
                 'Übung: ma Elopa',
                 'Übung: ma Italija',
                 'Übung: pali',
                 'Übung: toki mama',
                 'Ubung: kon tawa 🌬️',
                 'Grundrechenart',
                 'Mom',
                 'Jaki',
                 'Konsonant',
                 'Alice Salomon',
                 'Unterstützte Kommunikation']

exclude_pages = [slugify(item) for item in exclude_pages]

missing = 0
total_length = len(link_dataframe.index)
for index, row in link_dataframe.iterrows():
    link = row['link']
    title = row['title']

    if title in exclude_pages:
        continue
  
    info = load_and_save_plain_text(link, name=title)

    #add additional information to meta file
    if info:
        for key, value in info.items():
            link_dataframe.at[index,key] = value
    else:
        missing+=1

    print(f"Loaded ({index}/{total_length})")

print(f"Missing pages: {missing}")

ABBA ist eine schwedische Musikgruppe. Sie machen Pop·musik .
Loaded (0/4175)
ADAC ist eine Abkürzung. ADAC steht für: A llgemeiner D eutscher A utomobil- C lub e.
Loaded (1/4175)
ADHS ist eine Abkürzung. Die Abkürzung steht für: A ufmerksamkeits d efizit- H yperaktivitäts s törung
Loaded (2/4175)
AFAIK ist ein Ausdruck im Internet. AFAIK bedeutet soweit ich weiß .
Loaded (3/4175)
AFK ist eine englische Abkürzung. Die Abkürzung bedeutet: "Nicht an der Tastatur"
Loaded (4/4175)
Der AN-12 Unfall in Juba ist am 4. November 2015 gewesen. Das Flugzeug (mit dem Namen Antonow An-12) ist abgestürzt.
Loaded (5/4175)
Die Aarhus-Konvention ist ein Vertrag . In dem Vertrag geht es um den Umwelt·schutz .
Loaded (6/4175)
Aaron Fotheringham ist ein Roll·stuhl·sportler. Er ist sehr bekannt.
Loaded (7/4175)
Eine Ab·bruch·kante ist das Ende von einem Hügel. Das heißt, an diesem Ende geht es berg·ab.
Loaded (8/4175)
Abend·land nennt man einen Teil von der Welt. Und zwar die Länder im Westen von Europa .


Loaded (147/4175)
Eine Anlauttabelle wird zum Lernen benutzt. Darauf sind Buchstaben und Bilder.
Loaded (148/4175)
Anna Faris ist eine amerikanische Schauspielerin . Und eine Sängerin.
Loaded (149/4175)
Annalena Baerbock ist eine Politikerin . Sie ist in der Partei die Grünen .
Loaded (150/4175)
Eine Person ist anonym. Also weiß niemand: Wer ist diese Person?
Loaded (151/4175)
anonym bedeutet namenlos. Anonymität ist das dazugehörige Hauptwort .
Loaded (152/4175)
Die Anonymous-Gruppe sind verschiedene Menschen, die zusammen gegen bestimmte Sachen protestieren . Zum Beispiel gegen Zensur .
Loaded (153/4175)
Bei einer Anorexie hat man keine Lust etwas zu essen. Wenn die Anorexie länger geht, verschwindet auch der Hunger .
Loaded (154/4175)
Ein Anrainer ist ein Nachbar. Das heißt: jemand wohnt neben dran
Loaded (155/4175)
Der Anschlag in Istanbul ist am 1. Januar 2017 gewesen. Dabei sind 37 Menschen getötet.
Loaded (156/4175)
Der Anschlag in Istanbul ist am 10. Dezember 2016 gewesen. Dabe

Loaded (286/4175)
Das Außen·ohr ist ein Teil vom Ohr . Zum Außen·ohr gehört die Ohr·muschel der Gehör·gang .
Loaded (287/4175)
Außenpolitik ist eine bestimmte Art von Politik . Bei der Außenpolitik geht es um die Beziehung zwischen dem eigenen Land und anderen Ländern.
Loaded (288/4175)
Das Wort Avatar kann verschiedenen Bedeutungen haben. Zum Beispiel: • Ein Avatar ist ein Bild von einer Person - Avatar (Bild) • Avatar ist ein Film Avatar (Film) • Avatar ist ein Kurs - Avatar (Kurs) • Avatar ist ein Zeichen·trick·film - Avatar (Trick·film) • Avatar ist ein Computer·spiel - Avatar (Spiel) • Avatar ist eine Musik·gruppe Avatar (Band)
Loaded (289/4175)
Ein Avatar ist ein Bild. Auf dem Bild sieht man eine Person.
Loaded (290/4175)
Aya Iwamoto ist eine Übersetzerin Aya Iwamoto ist eine der ersten Menschen mit Trisomie 21, die einen Universitätsabschluß haben.
Loaded (291/4175)
Aylan Kurdi ist ein Kind . Er war ein Flüchtling und ist 3 Jahre alt geworden.
Loaded (292/4175)
B2K ist eine Abkü

Bigotterie ist, wenn man etwas vorspielt. Wenn man so macht, als ob man an etwas glaubt.
Loaded (428/4175)
Ein Bikini ist ein Badeanzug für Frauen und Mädchen. Auf dem Bild ist eine Frau.
Loaded (429/4175)
Bi·lateralität bedeutet 2·seitig. 2 Sachen machen etwas zusammen.
Loaded (430/4175)
Die Bilderberg-Konferenz ist ein Treffen. Es treffen sich Menschen die Macht haben.
Loaded (431/4175)
Beim Bilderbuch-Kino werden Bilder in einem Buch angeschaut. Gleichzeitig liest jemand anderes aus dem Buch vor.
Loaded (432/4175)
Das Wort Bild-lauf-leiste kommt aus der Computer-welt. Eine Bild-lauf-leiste ist zum steuern da.
Loaded (433/4175)
Ein Bildschirmschoner ist ein Computerprogramm .
Loaded (434/4175)
Auf einer Bildtafel sind viele Bilder. Alle Bilder sind auf 1 Seite.
Loaded (435/4175)
Das Wort Bildung kann verschiedene Bedeutungen haben. • Bildung heißt meistens: Wissen und Lernen: Bildung (Wissen) • Wenn man etwas baut.
Loaded (436/4175)
Bildung heißt Lernen. Und Bildung heißt Wissen.
Loa

Der Bundes·präsident ist das Ober·haupt von Deutschland . Er ist ein Mann oder eine Frau aus Deutschland.
Loaded (573/4175)
Die Bundespräsidentenwahl 2017 ist eine Wahl . Bei der Wahl wird der Bundes·präsident gewählt.
Loaded (574/4175)
Der Bundesrat hat nicht die gleichen Funktionen in der Schweiz und in Deutschland. Wir erklären zuerst, was der Bundesrat in Deutschland macht.
Loaded (575/4175)
Erklärung in Leichter Sprache Bild gesucht
Loaded (576/4175)
Der Bundesrat hat Mitglieder aus den Landtagen der Bundesländer. Er ist die Vertretung der Bundesländer in Berlin.
Loaded (577/4175)
Die Bundesregierung von Deutschland besteht aus dem Bundeskanzler . Und seinen Bundesministern .
Loaded (578/4175)
Die Bundes·tags·fraktion ist eine Gruppe von Politiker . Es ist eine Fraktion .
Loaded (579/4175)
Der Bundes·tags·präsident arbeitet im Deutschen Bundes·tag . Er achtet darauf, dass sich alle im Bundestag an die Regeln halten.
Loaded (580/4175)
Die Bundes·tags·wahl ist eine Wahl in Deutschla

...

Die UN-Antifolterkonvention ist eine Abmachung. Viele Länder haben diese Abmachung unterschrieben.
Loaded (3576/4175)
Eine Konvention ist eine Übereinkunft. Die UNO hat einen Vertrag gemacht.
Loaded (3577/4175)
Bei der UN-Klima·konferenz treffen sich Politiker . Die Politiker kommen aus verschiedenen Ländern.
Loaded (3578/4175)
Die UN-Klima·konferenz in Paris 2015 ist ein Treffen. Es treffen sich Vertreter von 195 Länder.
Loaded (3579/4175)
UNESCO ist der Name von einer Organisation. Auf deutsch heißt die Organisation: Organisation der Vereinten Nationen für Erziehung, Wissenschaft und Kultur.
Loaded (3580/4175)
Der US-Dollar ist eine Währung . Währung ist Geld, das man in einem Land benutzt um zu bezahlen.
Loaded (3581/4175)
Die USA sind ein Land in Nord·amerika . Auf dem Bild ist eine Welt·karte.
Loaded (3582/4175)
Ein USB-Stick ist ein kleines Speicher·gerät für den Computer . Auf dem Bild ist ein gelber USB-Stick.
Loaded (3583/4175)
Ubuntu Linux ist ein Betriebs·system . Mit Ubuntu

Vonnöten bedeutet dass etwas "nötig" ist.
Loaded (3730/4175)
Eine Vorankündigung ist eine frühe Nachricht.
Loaded (3731/4175)
Ein Vorarbeiter ist der Chef von Arbeitern. Er steht zwischen den Arbeitern und anderen Chefs.
Loaded (3732/4175)
Der Vorfall im Thalys-Zug 9364 war am 21. August 2015. Der Vorfall ist während einer Zug·fahrt passiert. Ein Mann hat im Zug auf andere Menschen geschossen.
Loaded (3733/4175)
Ein Vorkoster probiert Essen und Trinken. Das macht der Vorkoster bevor es jemand anderes macht.
Loaded (3734/4175)
Ein Vorort ist ein Ort vor einer Stadt. Es gibt Häuser und Wohnungen.
Loaded (3735/4175)
Mit Vorrats·daten·speicherung ist ein Gesetz gemeint. Das Gesetz ist für Telefon·firmen und Internet·firmen.
Loaded (3736/4175)
Ein Vorreiter ist jemand, der etwas Neues als erstes macht.
Loaded (3737/4175)
Vorschrift-Zeichen sind Verkehrs-Schilder . Alle Vorschrift-Zeichen haben eine Nummer.
Loaded (3738/4175)
Eine Vorsorge·vollmacht ist ein Schriftstück. Da steht drauf, dass

Loaded (3872/4175)
Ein Winkeladvokat ist eine Person. Diese Person arbeitet ähnlich wie ein Anwalt.
Loaded (3873/4175)
Wenn es plötzlich kalt wird. Und plötzlich Schnee fällt, nennt man das Winter-einbruch .
Loaded (3874/4175)
Wenn man im Winter dicker wird ist das der Winterspeck . Das heißt: Winterspeck ist mehr Körper-gewicht .
Loaded (3875/4175)
Bei der Winterzeit wird die Uhr 1 Stunde zurück gestellt. Das bedeutet: dass diese Nacht 1 Stunde länger ist, als die anderen Nächte.
Loaded (3876/4175)
Wirtschaft ist alles was Menschen mit Geld machen. Damit es den Menschen gut geht.
Loaded (3877/4175)
Das Wort Wirtschaft kann verschiedenen Bedeutungen haben. Zum Beispiel: • Ökonomie ist ein anderes Wort für Wirtschaft.
Loaded (3878/4175)
Beim Wirtschafts·kreislauf gibt es den Staat . Und es gibt Unternehmen .
Loaded (3879/4175)
Wissen sind Informationen. Alles was man genau kennt, ist Wissen.
Loaded (3880/4175)
Wladimir Klitschko ist ein ehemaliger Boxer. Er war mehrfacher Welt·meister i

Ein Aspirational Spender ist eine Person, die sich etwas kauft. Weil sie damit später etwas machen will.
Loaded (4016/4175)
Ein Auto·didakt ist eine Person. Sie bringt sich selbst etwas bei.
Loaded (4017/4175)
Ein Avatar ist ein Bild. Auf dem Bild sieht man eine Person.
Loaded (4018/4175)
Basale Stimulation soll den ganzen Körper anregen. Basale Stimulation ist für Menschen mit einer schweren Behinderung.
Loaded (4019/4175)
Betreuungsgeld ist Geld . Das Geld bekommt eine Familie für ihre Kinder.
Loaded (4020/4175)
Beim Bierbrauen stellt man Bier her. Bier·brauen macht man in einer Brauerei .
Loaded (4021/4175)
Binge Watching ist sehr langes Fernseh·schauen. Zum Beispiel einen ganzen Tag lang nur Fernseh schauen.
Loaded (4022/4175)
Biografie-arbeit ist ein Weg dazu: Sich selber besser kennen zu lernen. Eine Rückschau auf sein eigenes Leben dient dazu: Sich seines vergangenem Lebens bewusst zu werden.
Loaded (4023/4175)
Die Bundestagswahl 2021 ist eine Bundes·tags·wahl . Die Bundes·tags·

Termin·absage heißt: Ein Mensch sagt einen Termin ab. Bild gesucht
Loaded (4134/4175)
Erklärung in Leichter Sprache Bewerbung um einen Ausbildungsplatz als Mechatroniker
Sehr geehrter Herr Schmeckenbecher,
mein Berufswunsch ist Mechatroniker.                                                                                                            Ich möchte Mechatroniker werden, weil mich die Kombination von Mechanik und Elektronik anspricht und weil ich überzeugt bin, nach der Ausbildung eine abwechslungsreiche und spannende Tätigkeit zu finden.
Loaded (4135/4175)
Therapie·tiere sind Tiere . Therapie·tiere helfen Menschen.
Loaded (4136/4175)
Tonies ist der Name von einer Firma. Die Firma baut Tonie·boxen.
Loaded (4137/4175)
Menschen die transsexuell sind empfinden so: Sie sind mit dem falschen Geschlecht geboren. Sie gehören eigentlich dem anderen Geschlecht an.
Loaded (4138/4175)
Die Treuhandanstalt war eine Anstalt . Sie soll sich um Firmen in der DDR kümmern.
Loaded (4139/4175)
Ei

#Save scraped data

In [62]:
!zip -r corpus.zip plain_text -j
link_dataframe.to_csv('meta.csv', index=False)

updating: aids.txt (deflated 18%)
updating: totensonntag.txt (deflated 29%)
updating: david-bowie.txt (deflated 12%)
updating: idee.txt (deflated 9%)
updating: unter-einer-decke-stecken.txt (deflated 27%)
updating: imho.txt (deflated 17%)
updating: echelon.txt (deflated 15%)
updating: lobby-politik.txt (deflated 9%)
updating: striptease.txt (deflated 2%)
updating: platzverweis.txt (deflated 17%)
updating: big-mac.txt (deflated 5%)
updating: sergey-brin.txt (deflated 11%)
updating: tora.txt (deflated 11%)
updating: patio.txt (stored 0%)
updating: patzer.txt (deflated 21%)
updating: ubung-institut.txt (deflated 25%)
updating: botanischer-garten-karlsruhe.txt (deflated 16%)
updating: gesetz.txt (deflated 13%)
updating: growetterlage.txt (deflated 26%)
updating: welttag-des-schneemanns.txt (deflated 10%)
updating: leistungskurs.txt (deflated 20%)
updating: inas-fuballweltmeisterschaft-2014.txt (deflated 20%)
updating: tag-gegen-rassismus.txt (deflated 27%)
updating: gez.txt (stored 0%)
upd

updating: roofer.txt (deflated 14%)
updating: lesen.txt (deflated 19%)
updating: hot-swap.txt (deflated 13%)
updating: rikscha.txt (stored 0%)
updating: monarchie.txt (stored 0%)
updating: schneewittchen.txt (deflated 1%)
updating: arecibo-botschaft.txt (deflated 3%)
updating: bratwurst.txt (deflated 25%)
updating: konjunktiv.txt (deflated 29%)
updating: shitstorm.txt (stored 0%)
updating: ubungverkehrsstau.txt (deflated 8%)
updating: selbstbestimmung.txt (deflated 2%)
updating: bibliophilie.txt (deflated 13%)
updating: restaurationsbrot.txt (deflated 11%)
updating: ariana-grande.txt (deflated 2%)
updating: gated-community.txt (deflated 13%)
updating: festbeleuchtung.txt (deflated 7%)
updating: semantisch-lexikalische-storung.txt (deflated 14%)
updating: leo-tolstoi.txt (stored 0%)
updating: konigsteiner-schlussel.txt (deflated 16%)
updating: anschlag-in-stockholm-2017.txt (deflated 24%)
updating: blindganger.txt (deflated 4%)
updating: attentat.txt (deflated 9%)
updating: walpurgisnac

updating: abendland.txt (deflated 7%)
updating: biologie.txt (deflated 13%)
updating: weltnichtrauchertag.txt (deflated 17%)
updating: ubung-new-work.txt (deflated 2%)
updating: ross-und-reiter-nennen.txt (deflated 20%)
updating: berufsgenossenschaft.txt (deflated 29%)
updating: chanukka.txt (stored 0%)
updating: csu.txt (deflated 11%)
updating: gaokao.txt (deflated 4%)
updating: audiodeskription.txt (deflated 23%)
updating: hagia-sophia.txt (deflated 9%)
updating: hochwasser.txt (deflated 20%)
updating: squash.txt (stored 0%)
updating: behindertengleichstellungsgesetz.txt (deflated 16%)
updating: dornroschen.txt (deflated 6%)
updating: elbphilharmonie.txt (deflated 5%)
updating: termin.txt (deflated 4%)
updating: naturgesetz.txt (deflated 13%)
updating: retour.txt (stored 0%)
updating: grammatische-storung.txt (deflated 16%)
updating: hefe.txt (deflated 8%)
updating: tween.txt (deflated 13%)
updating: leitartikel.txt (deflated 15%)
updating: konflikt.txt (deflated 13%)
updating: minar

updating: raumfahrer.txt (deflated 18%)
updating: internationale-standardbuchnummer.txt (deflated 14%)
updating: paralympics-rudern.txt (stored 0%)
updating: moonalphabet.txt (deflated 15%)
updating: ebook-reader.txt (deflated 4%)
updating: europaisches-parlament.txt (deflated 33%)
updating: ramsch.txt (deflated 14%)
updating: konzentrationslager.txt (stored 0%)
updating: staat.txt (stored 0%)
updating: transgender.txt (stored 0%)
updating: dbt.txt (deflated 8%)
updating: nurnberger-gesetze.txt (stored 0%)
updating: visegrad-gruppe.txt (deflated 21%)
updating: digital-subscriber-line.txt (deflated 27%)
updating: pompon.txt (deflated 9%)
updating: ubung-soziale-schicht.txt (deflated 13%)
updating: zweiter-arbeitsmarkt.txt (deflated 17%)
updating: holzmedien.txt (deflated 8%)
updating: deutsche-dogge.txt (deflated 9%)
updating: thora-heilige-schrift.txt (deflated 15%)
updating: abrollschuh.txt (deflated 13%)
updating: redestab.txt (deflated 21%)
updating: rentner.txt (deflated 5%)
updati

In [63]:
import os
import csv

# Dictionary to store name-content pairs
name_content_dict = {}

# Path to the directory containing the text files
files_directory = 'plain_text/'

# Path to the CSV file containing the names
csv_file_path = 'meta.csv'

# Read names from the CSV file
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    
    next(csv_reader)
    
    # Assuming names are in the first column
    name_list = [row[0] for row in csv_reader]

# Iterate through the list of names
for name in name_list:
    # Construct the file path for each name
    file_path = os.path.join(files_directory, f'{name}.txt')

    # Check if the file exists
    if os.path.exists(file_path):
        # Read the content of the file
        with open(file_path, 'r') as file:
            content = file.read()

        # Add the name-content pair to the dictionary
        name_content_dict[name] = content
    else:
        # If the file doesn't exist, set the value to None or handle accordingly
        name_content_dict[name] = None

# Print the resulting dictionary
print(name_content_dict)

{'abba': 'ABBA ist eine schwedische Musikgruppe. Sie machen Pop·musik .', 'adac': 'ADAC ist eine Abkürzung. ADAC steht für: A llgemeiner D eutscher A utomobil- C lub e.', 'adhs': 'ADHS ist eine Abkürzung. Die Abkürzung steht für: A ufmerksamkeits d efizit- H yperaktivitäts s törung', 'afaik': 'AFAIK ist ein Ausdruck im Internet. AFAIK bedeutet soweit ich weiß .', 'afk': 'AFK ist eine englische Abkürzung. Die Abkürzung bedeutet: "Nicht an der Tastatur"', 'an-12-unfall-in-juba': 'Der AN-12 Unfall in Juba ist am 4. November 2015 gewesen. Das Flugzeug (mit dem Namen Antonow An-12) ist abgestürzt.', 'aarhus-konvention': 'Die Aarhus-Konvention ist ein Vertrag . In dem Vertrag geht es um den Umwelt·schutz .', 'aaron-fotheringham': 'Aaron Fotheringham ist ein Roll·stuhl·sportler. Er ist sehr bekannt.', 'abbruchkante': 'Eine Ab·bruch·kante ist das Ende von einem Hügel. Das heißt, an diesem Ende geht es berg·ab.', 'abendland': 'Abend·land nennt man einen Teil von der Welt. Und zwar die Länder im

In [64]:
import json

# Path to the new CSV file
output_json_path = 'dict.json'

#Write the dictionary to a new JSON file
with open(output_json_path, 'w') as json_file:
    json.dump(name_content_dict, json_file, indent=2)

print(f'The JSON file has been created at: {output_json_path}')

The JSON file has been created at: dict.json
